In [21]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt

In [22]:
from qubap.qiskit.hamiltonians import test_hamiltonian
from qubap.qiskit.variational_algorithms import classical_solver

ImportError: cannot import name 'BasicAer' from 'qiskit' (/Users/khen/repos/NNBP/venv/lib/python3.9/site-packages/qiskit/__init__.py)

In this tutorial we are going to implement a VQE with all the features of our module, that is, an pretrained adiabatic VQE with State Efficient Ansatz. We will use the following 10-qubits hamiltonian 

In [ ]:
num_qubits = 10
H = test_hamiltonian( num_qubits )
print(H)

In [ ]:
exact_min_energy = classical_solver(H).eigenvalue
exact_min_energy

First we are going to do a standard VQE with `EfficientSU2` with a single layer of entangling gates as parametric circuit.

In [ ]:
from qiskit.circuit.library import EfficientSU2
from qubap.qiskit.variational_algorithms import VQE, energy_evaluation 
from qiskit_aer import AerSimulator

In [ ]:
num_reps = 1
ansatz = EfficientSU2( num_qubits, ['ry','rz'], 'circular', num_reps).decompose()

We start the search from a random point.

In [ ]:
np.random.seed(102)
initial_guess = np.random.randn( ansatz.num_parameters )*np.pi
num_iters     = 300
backend       = AerSimulator( shots=2**8 )

In [ ]:
results_vqe = VQE( H, ansatz, initial_guess, num_iters, backend )
analitic_results_vqe = [ energy_evaluation( H, ansatz, x, AerSimulator(method='statevector') ) for x in results_vqe['x'] ]

We obtain the following results. Clearly, this problem exhibits a barren plateau.  

In [ ]:
plt.plot( analitic_results_vqe )

Now, we create the local Hamiltonian.

In [ ]:
from qubap.qiskit.cost_function_barren_plateau import global2local

In [ ]:
H_local = global2local( H )
print( H_local )

Next, perform the pretraining with matrix product states.

In [ ]:
from qubap.qiskit.mps_pretraining import Ansatz, VQE_pretrain

In [ ]:
ansatz_mps = Ansatz( num_qubits, diagonal=True )

In [ ]:
num_iters_train = 300
results_pretrain = VQE_pretrain( H, num_iters_train )

Finally, we construct the state-efficient ansatz. Notice that we compose the state-efficient ansatz with the optimal MPS ansatz in order to start the optimization from the pretraining. 

In [ ]:
from qubap.qiskit.state_efficient_ansatz import ansatz_constructor 
num_reps_sea = 1
ansatz_sea   = ansatz_constructor( num_qubits, deep=[num_reps_sea,num_reps_sea,num_reps_sea]).decompose()
ansatz_sea.draw( 'mpl', fold=-1 )

In [ ]:
ansatz_sea_mps = ansatz_sea.compose( ansatz_mps.bind_parameters(results_pretrain['x'][-1]) ).decompose()
ansatz_sea_mps.draw( 'mpl', fold=-1 )

Executing the adiabatic VQE.

In [ ]:
from qubap.qiskit.variational_algorithms import VQE_adiabatic

In [ ]:
initial_guess = np.zeros( ansatz_sea_mps.num_parameters ) + 0.01
results_adiabatic = VQE_adiabatic( H_local, H, ansatz_sea_mps, initial_guess, num_iters, backend )

In [ ]:
analitic_results_adiabatic   = [ energy_evaluation( H, ansatz_sea_mps, x, AerSimulator(method='statevector') ) for x in results_adiabatic['x'] ]

We can see that the combined routine is able of outperformed the barren plateau.

In [ ]:
plt.plot( analitic_results_vqe )
plt.plot( analitic_results_adiabatic )
plt.legend(['Standard VQE', 'Combined method'])

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table